In [1]:
import os
import re

from langchain_community.document_loaders.confluence import ConfluenceLoader
from dotenv import load_dotenv
from typing import (Any,List)

In [2]:
load_dotenv()
persist_directory = "./chroma_db/"
space_key = "IBW"
confluence_url = os.getenv("CONFLUENE_URL")
username = os.getenv("CONFLUENCE_USERNAME")
api_key = os.getenv("CONFLUENCE_API_KEY")

In [21]:
from langchain_community.document_loaders.confluence import ContentFormat

## 1. Extract the documents
loader = ConfluenceLoader(
    #content_format=ContentFormat.STORAGE,
    url=confluence_url,
    username = username,
    api_key= api_key
)

documents = loader.load(
    space_key=space_key,
    #page_ids= [25397420, 25398513], # [26706543, 185500205], # 398098434 # bankens = 25398483, 25398513
    include_attachments=False,
    include_comments=False,
    limit=100,
    max_pages=1000,
    keep_markdown_format=True,
    )

In [22]:
from splitters_confluence import ConfluenceMarkdownChunker
from langchain_text_splitters.markdown import MarkdownTextSplitter, MarkdownHeaderTextSplitter

header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "level_1"), ("##", "level_2"), ('###', "level_3")]
)
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)

chunker = ConfluenceMarkdownChunker(markdown_header_splitter=header_splitter, markdown_text_splitter=text_splitter)

In [23]:
final = chunker.chunker(documents)

In [24]:
#for d in final:
#    print("~"*10)
#    print(len(d.page_content)," : ", d.page_content[:100])
#    print("d.page_content[-100:]: ",d.page_content[-100:])
#    print("d.metadata['source']: ", d.metadata["source"])
#    print('d.metadata["sub-title"]: ',d.metadata["sub-title"])


# Test Retrieval

In [33]:
from langchain_community.embeddings import GPT4AllEmbeddings
#from langchain_community.vectorstores import Chroma
#from langchain.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS

In [26]:
model_name = "BAAI/bge-large-en-v1.5"#"sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 614kB/s]
README.md: 100%|██████████| 94.6k/94.6k [00:00<00:00, 657kB/s]
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 616kB/s]
config.json: 100%|██████████| 779/779 [00:00<00:00, 9.18MB/s]
model.safetensors: 100%|██████████| 1.34G/1.34G [12:20<00:00, 1.81MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 832kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 915kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 806kB/s]
1_Pooling/config.json: 100%|██████████| 191/191 [00:00<00:00, 1.16MB/s]


In [35]:
vectordb = FAISS.from_documents(final, embedding=embeddings)

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
INFO:chromadb.db.duckdb:Exiting: Cleaning up .chroma directory


In [36]:
question = "Spärra bankkort"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='**Spärra kort** Om kunden inte vill/kan reklamera via app kan den skicka in blankett. Blanketten finns att skriva ut på vår hemsida. Om kunden inte har möjlighet att skriva ut den kan vi beställa den på US45 under kategorin VAKUN i Ricard. Blanketten heter\xa0**Reklamationsblankett Bank**.\nBanken har ingen möjlighet att i efterhand upphäva en spärr på ett kort. **Kortet kan spärras av kortinnehavaren eller annan person, kunden behöver inte vara identifierad**. Spärras ett kort så spärras alla versioner av det kortet, har kunden alltså kvar en nyare version av kortet hemma går det inte att använda den versionen utan kunden behöver få ett helt nytt kort med nytt kortnummer.\nÄr version 2 av ett kort aktiverat går det inte att använda version 1 av kortet. Tappar kund bort kortet och aktiverar en senare version behöver inte den borttappade versionen spärras.\nSpärr i digitala kanaler**ICA Bankens app**', metadata={'title': 'Hantera kort', 'id': '25398483', 'source'

In [37]:
question = "Hur spärrar jag bankkort?"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='Allmän beskrivning/Frågor och svar Mobilt BankID Kund aktiverar Mobilt BankID, men det blir spärrat direkt, vad beror det på? Om en kund aktiverar ett Mobilt BankID, men det spärras direkt efter utgivning, beror det ibland på att det finns en spärr som medför att kunden inte kan ha ett Mobilt BankID.\nFör att kontrollera om det finns en spärr måste man kontakta AFC (skill Reklamation och Bedrägeri). Vi ska inte informera kund om att vi ska kontrollera ev spärr.\nFinns ingen spärr får man lägga en felanmälan enligt ordinarie rutin, med de uppgifter som behövs för att kunna felsöka (tidpunkt, typ av enhet, operativssytem, felmeddelanden osv)\n**OBS!** I de fall kunden får ett avslag/felmeddelande i ansökanflödet och inte kommer vidare alls, ska man alltid börja med att kontrollera att kunden uppfyller kraven för att ha MBID.\nTe x finns det en felkod på ansökan som kan ge en orsak till ett ev avslag, uppfyller kunden kraven för att ha MBID (medgivande finns, kunde

In [38]:
question = "Får jag skicka en personlig dosa utomlands via fullmakt?"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='**Möjliggör att man kan skicka brevledes uppdrag till oss. Det går inte att begära uppdrag via telefon eller säkert meddelande.** * Fullmakten ska visas upp i original när den används.\nHur utför man ett ärende med en framtidsfullmakt?1. Fullmaktshavaren måste skicka in ett skriftligt uppdrag till: *ICA Banken AB, 504 82, Borås*. Det går bra att skicka det med rekommenderat brev. Vi får inte ta emot uppdrag över telefon/säkert meddelande.\n2. Brevet ska tydligt beskriva vad som ska uträttas och innehålla fullmaktshavarens namn, personnummer och signatur.\n3. Framtidsfullmakten ska bifogas i **original** när ett uppdrag ska utföras. När uppdraget är utfört returnerar banken fullmakten i original med rekommenderat brev till fullmaktshavaren.\n4. Kopia på fullmaktshavarens ID-handling ska bifogas.\n5. Banken **tar inte emot** framtidsfullmakter för arkivering. Fullmaktshavaren ansvarar för att kontakta banken vid de tillfällen som fullmakten ska användas.', metadat

In [39]:
question = "Kan man ha skyddat namn på Swish när man har skyddad identitet?"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='Frågor och svar Kan jag ansluta ett mobilnummer till mitt konto som någon annan äger, t ex mina föräldrar eller min arbetsgivare ?Ja, det fungerar.\nHur länge sparas historiken för mottagna och skickade betalningar?För svar på denna fråga, se Swish [hemsida](https://www.swish.nu/hjalp#vad_kan_vi_hjalpa_dig_med).\nKan kund med skyddad identitet swisha?Ja, kund med skyddad identitet kan Swisha. ICA Banken har inte någon inbyggd funktion att använda alias för kund i Swish-flödet. I bankens finstilta villkor har vi gjort ett val att ta med information under allmänna avsnittet för att t.ex. person med skyddad identitet själv ska kunna\xa0ta ställning om Swish-tjänsten är lämplig att ansluta sig till eller inte:\n"Betalningen med Swish kan göras av fysiska person till andra fysiska och juridiska personer som är anslutna till Swish. Vid användning av Swish får parterna i samband med en transaktion uppgift om varandras namn och mobilnummer".', metadata={'title': 'Swish'

In [40]:
question = "Jag har samordningsnummer, finns det några begrsäningar. ?"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='**Allmän beskrivning**/Frågor och svar #### faq\nVad kan och får kunder utan svenskt personnummer göra?x = vad kunden kan göra\nVad är ett samordningsnummer?Samordningsnummer är en identitetsbeteckning för personer som inte är eller har varit folkbokförda i Sverige. Syftet med samordningsnummer är att myndigheter och andra samhällsfunktioner ska kunna identifiera personer även om de inte är folkbokförda i Sverige.\nSamordningsnummer består liksom personnumret av tio siffror. De inledande sex siffrorna utgår från personens födelsetid med den skillnaden att man lägger till 60 till födelsedagen. För en person som är född den 23 augusti 1964 så blir de sex första siffrorna i samordningsnumret därför 640883.\n(Information hämtad från Skatteverkets hemsida.)', metadata={'title': 'Kortansökan utan svenskt personnummer', 'id': '25397360', 'source': 'https://ica.atlassian.net/wiki/spaces/IBW/pages/25397360', 'sub-title': '**Allmän beskrivning**/Frågor och svar'}),
 Docum

In [42]:
question = "felkod 65"
docs = vectordb.similarity_search(question)
docs

[Document(page_content='**Felsökning/reklamation** Felkod ACMT23Felkoden *ACMT23* betyder att kunden har max antal Swish-anslutningar redan (man kan ha max 3 st i sitt eget namn och max 3 till sina barn). Om kunden har Swish-anslutning i annan bank som ska avslutas behöver hen vända sig dit.\nReklamation av Swish-köpI och med att Swish likställs med kontant betalning kan man inte reklamera ett Swish-köp till oss. I det fall en kund vill reklamera ett Swish-köp, exempelvis om hen inte fått varan/tjänsten, hänvisar vi kunden till att reklamera hos företaget i fråga. Om det handlar om ett bedrägeri får kunden göra en polisanmälan.\nSök efter annan rutin i Wikinpage\ngreendottedInnehåll på sidan3\nLäs mer[Mobilt BankID](https://ica.atlassian.net/wiki/display/IBW/Mobilt+BankID)\n[Utredning betalning/överföring](https://ica.atlassian.net/wiki/pages/viewpage.action?pageId=25400186)\nInom kort: uppdaterat gränssnitt för Swish Online\n---\nDokumentägare: Produktspecialist Betala\nSenast faststä

In [43]:
question = "felkod 65"
embedding_vector = embeddings.embed_query(question)
docs_and_scores = vectordb.similarity_search_by_vector(embedding_vector)
docs

[Document(page_content='**Felsökning/reklamation** Felkod ACMT23Felkoden *ACMT23* betyder att kunden har max antal Swish-anslutningar redan (man kan ha max 3 st i sitt eget namn och max 3 till sina barn). Om kunden har Swish-anslutning i annan bank som ska avslutas behöver hen vända sig dit.\nReklamation av Swish-köpI och med att Swish likställs med kontant betalning kan man inte reklamera ett Swish-köp till oss. I det fall en kund vill reklamera ett Swish-köp, exempelvis om hen inte fått varan/tjänsten, hänvisar vi kunden till att reklamera hos företaget i fråga. Om det handlar om ett bedrägeri får kunden göra en polisanmälan.\nSök efter annan rutin i Wikinpage\ngreendottedInnehåll på sidan3\nLäs mer[Mobilt BankID](https://ica.atlassian.net/wiki/display/IBW/Mobilt+BankID)\n[Utredning betalning/överföring](https://ica.atlassian.net/wiki/pages/viewpage.action?pageId=25400186)\nInom kort: uppdaterat gränssnitt för Swish Online\n---\nDokumentägare: Produktspecialist Betala\nSenast faststä